In [1]:
import json
import cfscrape
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
# from fake_useragent import UserAgent
# useragent = UserAgent() 

In [2]:
# useragent.random 

In [3]:
 yandex_all_id = 1163323346,1243520916,1261357675,1503884741,1084635139,1168675286,68938452823,1142895335,46866695409,1156420979,1230403863,1808210910,1137049971,1173855410,1737906159,8021941936,48923682156,106396488794,1219578186,1137893690,147075068049,1395617057,1136669337,1169237983,1209203365,1187228400,1199923913,155391100274,1249340350,95798602015,1092610521,222501363394,1109319679,1170778562,1357472078,111881168077,1315558370,135157708122,1171367346,1004521162,175499965097,1082278919,39505328349,181349390806,57361398610,238385424377,217576127302,174583421422,238345237585,1761503433,1092543151,1146856379,129281957669,58618253938

In [4]:
#yandex_all_id = 1199923913,1146856379,1752726383,68938452823,1136669337,48923682156,1084785161,106396488794,1169237983,1171367346,244588988530,95798602015,1808210910,74755559887,222108751806,1004521162,238385424377,1178994977,1809097327


In [5]:
#yandex_all_id = 1146856379,1752726383,68938452823,1136669337,48923682156,1084785161,106396488794,1169237983,1171367346
#122953255163

In [6]:
Data_id =  pd.read_excel('Data_id.xlsx',engine = 'openpyxl')  #pd.DataFrame((yandex_all_id), columns = ['id'])

In [10]:
Data_id.head(81)

,id
0,1218514905
1,1036543306
2,222108751806
3,59452068765
4,1166206684
...,...
76,1761503433
77,1092543151
78,1146856379
79,129281957669


In [11]:
len(Data_id)

81

In [12]:
#'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'

In [13]:
estimates_all = pd.DataFrame()

In [14]:
for i in range(1):
    yandex_id = str(Data_id.iloc[i]['id'])
    date_object = time.strftime("%d.%m.%Y")
    time_object = time.strftime("%H:%M:%S")
    ya = pd.DataFrame({"date": date_object,"time":time_object, "id": yandex_id}, index = [1])
    ya['key'] = 1
    def get_count_star(review_stars):
        star_count = 0
        for review_star in review_stars:
            if '_empty' in review_star.get('class'):
                continue
            elif '_half' in review_star.get('class'):
                star_count = star_count + 0.5
            else:
                star_count = star_count + 1
        return star_count

    def get_session():
        session = requests.Session()
        session.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
            'upgrade-insecure-requests': '1',
            #'user-agent': useragent.random,
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
            'viewport-width': '1920'
        }
        return cfscrape.create_scraper(sess=session)

    def parsing_data(data_result, r):
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            reviews = soup.find('div', {"data-chunk": "reviews"})
        except:
            return None
        try:
            summary_rating__main = reviews.find('div', {"class": "business-summary-rating__main-rating"})
        except:
            summary_rating__main = None
        try:
            rank_summary_rating__main = summary_rating__main.find('div',{"class": "business-summary-rating-badge-view__rating"}).text
        except:
            rank_summary_rating__main = None
        try:
            count_summary_rating__main = summary_rating__main.find('div', {
                "class": "business-summary-rating-badge-view__rating-count"}).text
        except:
            count_summary_rating__main = None
        try:
            stars_summary_rating__main = get_count_star(
                summary_rating__main.find('div', {"class": "business-rating-badge-view__stars"}))
        except:
            stars_summary_rating__main = None
        try:
            address = soup.find('meta', {"itemprop": "address"}).get('content')
        except:
            address = None
        
        try:
            site = soup.find('div', {"class": "business-urls-view__url"}).text
        except:
            site = None
        
        try:
            Reviews = soup.find('div', {"class": "tabs-select-view__title _name_reviews"}).text
        except:
            Reviews = None
        
        try:
            name = soup.find('h1',{"class":"orgpage-header-view__header","itemprop":"name"}).text
        except:
            return None
           
            
        
        data_result['company_info'] = {
            'company_rating': rank_summary_rating__main,
            'company_count_reviews': count_summary_rating__main,
            'company_count_stars': stars_summary_rating__main,
            'company_address': address,
            'company_name': name,
            'site': site,
            'Reviews': Reviews
        }
        return data_result
    
    url = 'https://yandex.ru/maps/org/letay/' + yandex_id
    session = get_session()
    r = session.get(url)
    data_result = {
        'company_info': {}
    }
    data_result
    data_result = parsing_data(data_result, r)
    df = pd.DataFrame(data_result)
    addr = df.loc['company_address']['company_info']
    name = df.loc['company_name']['company_info']
    while addr == None:
        time_new = time.strftime("%H:%M:%S")
        print(time_new,' New query for ', name, url)
        time.sleep(300)
        session = get_session()
        r = session.get(url)
        data_result = {
            'company_info': {}
        }
        data_result
        data_result = parsing_data(data_result, r)
        df = pd.DataFrame(data_result)
        addr = df.loc['company_address']['company_info']
    df = df.transpose()   
    if len(data_result['company_info']) == 0:
        data_result = None
    with open('company_reviews.json', 'w', encoding='utf-8') as f:
        json.dump(data_result, f, ensure_ascii=False, indent=4)
    print(i,' ',time_object,' ', name, ' ID:',yandex_id,' ',url,' - Success')
    df['key'] = 1
    df['url'] = url
    df = ya.merge(df, on = 'key', indicator=True)
    estimates_all = pd.concat([estimates_all,df])
    time.sleep(7+i)

0   09:02:10   Летай  ID: 1218514905   https://yandex.ru/maps/org/letay/1218514905  - Success


In [15]:
data_result

{'company_info': {'company_rating': 'Рейтинг\xa04,5',
  'company_count_reviews': '29 оценок',
  'company_count_stars': 4.5,
  'company_address': 'Республика Татарстан, Кайбицкий район, село Большие Кайбицы, Солнечный бульвар, 11',
  'company_name': 'Летай',
  'site': 'letai.ru',
  'Reviews': 'Отзывы11'}}

In [16]:
estimates_all

,date,time,id,key,Reviews,company_address,company_count_reviews,company_count_stars,company_name,company_rating,site,url,_merge
0,12.02.2024,09:02:10,1218514905,1,Отзывы11,"Республика Татарстан, Кайбицкий район, село Бо...",29 оценок,4.5,Летай,"Рейтинг 4,5",letai.ru,https://yandex.ru/maps/org/letay/1218514905,both


In [17]:
data_result

{'company_info': {'company_rating': 'Рейтинг\xa04,5',
  'company_count_reviews': '29 оценок',
  'company_count_stars': 4.5,
  'company_address': 'Республика Татарстан, Кайбицкий район, село Большие Кайбицы, Солнечный бульвар, 11',
  'company_name': 'Летай',
  'site': 'letai.ru',
  'Reviews': 'Отзывы11'}}

In [15]:
day = time.strftime("%d%m%Y_%H%M")

In [16]:
#estimates_all.to_excel('D:\\estimates\estimates_all_'+day+'.xlsx',index = False)

In [17]:
#estimates_all.to_csv('D:\\estimates\estimates_all_'+day+'.csv',sep=';', encoding='cp1251')

In [18]:
#df.to_csv(file_name, sep='\t', encoding='utf-8')